In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import pandas as pd
import os
import torch.optim as optim
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import torch
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA está disponível. Dispositivo configurado para GPU.")
else:
    device = torch.device("cpu")
    print("CUDA não está disponível. Usando CPU.")

CUDA não está disponível. Usando CPU.


Extraindo os dados

In [ ]:
zip_path = '/content/drive/MyDrive/Pesquisas e Projetos/BirdCLEF/Espectograms.zip'
extract_to = '/content/'
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f'Arquivos extraídos para {extract_to}')

Arquivos extraídos para /content/


In [ ]:
zip_path = '/content/drive/MyDrive/Pesquisas e Projetos/BirdCLEF/birdclef-2024_Test.zip'
extract_to = '/content/BirdCLEF'
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f'Arquivos extraídos para {extract_to}')

Arquivos extraídos para /content/BirdCLEF


Renomeando as imagens

In [ ]:
base_dir = '/content/Espectograms/espectrogramasMEL_train_audio'
def rename_files(base_dir):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith("_espectrograma_mel.png"):
                # Extrair a parte do nome do arquivo que queremos manter
                new_name = file.split('_espectrograma_mel')[0] + '.png'
                # Caminho completo para o arquivo antigo e novo
                old_path = os.path.join(root, file)
                new_path = os.path.join(root, new_name)
                os.rename(old_path, new_path)
                print(f'Renamed: {old_path} to {new_path}')

rename_files(base_dir)

Renamed: /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC288101_espectrograma_mel.png to /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC288101.png
Renamed: /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC329165_espectrograma_mel.png to /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC329165.png
Renamed: /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC239744_espectrograma_mel.png to /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC239744.png
Renamed: /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC771306_espectrograma_mel.png to /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC771306.png
Renamed: /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC788753_espectrograma_mel.png to /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC788753.png
Renamed: /content/Espectograms/espectrogramasMEL_train_audio/ashdro1/XC396854_espectrograma_mel.png to /content/Espectog

### Visualizando as tabelas

In [ ]:
df1 = pd.read_csv("/content/BirdCLEF/birdclef-2024_Test/eBird_Taxonomy_v2021.csv", delimiter=';')
df1

,TAXON_ORDER,CATEGORY,SPECIES_CODE,PRIMARY_COM_NAME,SCI_NAME,ORDER1,FAMILY,SPECIES_GROUP,REPORT_AS
0,1,species,ostric2,Common Ostrich,Struthio camelus,Struthioniformes,Struthionidae (Ostriches),Ostriches,NaN
1,6,species,ostric3,Somali Ostrich,Struthio molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
2,7,slash,y00934,Common/Somali Ostrich,Struthio camelus/molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
3,8,species,grerhe1,Greater Rhea,Rhea americana,Rheiformes,Rheidae (Rheas),Rheas,NaN
4,14,species,lesrhe2,Lesser Rhea,Rhea pennata,Rheiformes,Rheidae (Rheas),NaN,NaN
...,...,...,...,...,...,...,...,...,...
16748,34694,species,slcgro1,Slate-colored Grosbeak,Saltator grossus,Passeriformes,Thraupidae (Tanagers and Allies),NaN,NaN
16749,34697,species,bltgro2,Black-throated Grosbeak,Saltator fuliginosus,Passeriformes,Thraupidae (Tanagers and Allies),NaN,NaN
16750,34698,spuh,saltat1,saltator sp.,Saltator sp.,Passeriformes,Thraupidae (Tanagers and Allies),NaN,NaN
16751,34699,spuh,passer1,passerine sp.,Passeriformes sp.,Passeriformes,NaN,Others,NaN


In [ ]:
len(df1["SPECIES_CODE"].unique())

16753

In [ ]:
df2 = pd.read_csv("/content/BirdCLEF/birdclef-2024_Test/sample_submission.csv", delimiter=';')
df2

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,soundscape_1446779_5,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,soundscape_1446779_10,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,soundscape_1446779_15,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [ ]:
len(df2.columns.unique())

183

In [ ]:
df3 = pd.read_csv("/content/BirdCLEF/birdclef-2024_Test/train_metadata.csv", delimiter=';')
df3 = df3.dropna(how='all')
df3

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],392.297,1.181.987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.403,1.046.401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],363.319,1.273.555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],211.697,706.005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],155.442,737.733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...
1043,litswi1,[],[''],-283.701,324.101,Apus affinis,Little Swift,Maarten Sluijter,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/803755,litswi1/XC803755.ogg
1044,litswi1,[],[''],299.672,-96.641,Apus affinis,Little Swift,Falco,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/806822,litswi1/XC806822.ogg
1045,litswi1,[],[''],-0.6113,345.465,Apus affinis,Little Swift,GABRIEL LEITE,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/810705,litswi1/XC810705.ogg
1046,litswi1,[],[''],-0.5462,345.596,Apus affinis,Little Swift,GABRIEL LEITE,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/810749,litswi1/XC810749.ogg


In [ ]:
df3.scientific_name.unique()

array(['Muscicapa dauurica', 'Dicrurus leucophaeus', 'Prinia socialis',
       'Ninox scutulata', 'Dicrurus aeneus', 'Plegadis falcinellus',
       'Prinia hodgsonii', 'Streptopelia senegalensis',
       'Phylloscopus magnirostris', 'Apus affinis'], dtype=object)

### Extrator de Características

Importando modelo

In [ ]:
# Modificar a rede para adicionar GAP e remover a camada FC
def modify_network(model):
    model.fc = torch.nn.Identity()
    features = list(model.children())[:-1]  # Remover a última camada totalmente conectada
    features.append(torch.nn.AdaptiveAvgPool2d(output_size=(1, 1)))     # Adicionar Global Average Pooling
    model = torch.nn.Sequential(*features)
    return model

resnet50 = models.resnet50(pretrained=True)
resnet50_modified = modify_network(resnet50)
#resnet50_modified.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 115MB/s]


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def get_image_embedding(root_path, model, transform):
    embeddings = []
    device = next(model.parameters()).device  # Obter o dispositivo do modelo

    for class_dir in os.listdir(root_path):
        class_path = os.path.join(root_path, class_dir)
        if os.path.isdir(class_path):
            for image in os.listdir(class_path):
                if image.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, image)
                    img = Image.open(img_path).convert("RGB")  # Convertendo para RGB
                    img_t = transform(img)
                    img_t = img_t.to(device)  # Mover o tensor para o dispositivo do modelo
                    batch_t = torch.unsqueeze(img_t, 0)
                    # Desabilita o cálculo de gradiente para economizar memória e aumentar a velocidade
                    with torch.no_grad():
                        resnet_features = model(batch_t)
                        path_id = os.path.join(class_dir, image)
                        embeddings.append({
                            "path": path_id,
                            "class": class_dir,
                            "resnet_features": resnet_features.cpu().numpy()  # Convertendo para numpy para facilitar o uso posterior
                        })

    return embeddings

def get_image_embedding_test(image_path, model, transform):
    embeddings = []
    device = next(model.parameters()).device  # Obter o dispositivo do modelo
    for image in os.listdir(image_path):
        if image.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(image_path, image)
            img = Image.open(img_path).convert("RGB")
            img_t = transform(img)
            img_t = img_t.to(device)  # Mover o tensor para o dispositivo do modelo
            batch_t = torch.unsqueeze(img_t, 0)
            # Desabilita o cálculo de gradiente para economizar memória e aumentar a velocidade
            with torch.no_grad():
                        resnet_features = model(batch_t)
                        embeddings.append({
                            "path": image,
                            "resnet_features": resnet_features.cpu().numpy()  # Convertendo para numpy para facilitar o uso posterior
                        })
    return embeddings

In [ ]:
image_path = '/content/Espectograms/espectrogramasMEL_train_audio'
data = get_image_embedding(image_path, resnet50_modified, transform)

In [ ]:
image_path_ = '/content/Espectograms/espectrogramasMEL_unlabeled_soundscapes'
data_unabeled_soundscapes = get_image_embedding_test(image_path_, resnet50_modified, transform)

In [ ]:
data[0]

{'path': 'ashdro1/XC114598.png',
 'class': 'ashdro1',
 'resnet_features': array([[[[0.3838624 ]],
 
         [[0.51760083]],
 
         [[0.47127506]],
 
         ...,
 
         [[0.34451303]],
 
         [[0.4395803 ]],
 
         [[0.31603485]]]], dtype=float32)}

In [ ]:
data_unabeled_soundscapes[0]

{'path': '982682388_espectrograma_mel.png',
 'resnet_features': array([[[[0.3775505 ]],
 
         [[0.60143685]],
 
         [[0.46595356]],
 
         ...,
 
         [[0.32889137]],
 
         [[0.39920837]],
 
         [[0.33212382]]]], dtype=float32)}

### Metadados

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Normalizando as colunas da latitude e longitude

In [ ]:
scaler = MinMaxScaler()
df3['latitude'] = df3['latitude'].str.replace('.', '').str.replace(',', '.').astype(float)
df3['longitude'] = df3['longitude'].str.replace('.', '').str.replace(',', '.').astype(float)
df3[['latitude', 'longitude']] = scaler.fit_transform(df3[['latitude', 'longitude']])

In [ ]:
len(df3)

1048

In [ ]:
len(data)

1048

Nomeando a coluna filename do dataframe para fazer a verificação ao iterar e unir os metadados com os resultados dos embeddings

In [ ]:
df3['filename'] = df3['filename'].str.replace('.ogg', '.png')

In [ ]:
# Criar um dicionário para mapear os filenames com os metadados
metadata_dict = df3.set_index('filename').to_dict('index')

for embedding in data:
    file_path = embedding['path']
    if file_path in metadata_dict:
        metadata = metadata_dict[file_path]
        embedding.setdefault('filename', file_path.split('/')[-1])
        # Adicionar os metadados
        embedding.update({
            'latitude': metadata['latitude'],
            'longitude': metadata['longitude'],
            'scientific_name': metadata['scientific_name'],
            'common_name': metadata['common_name']
        })

for embedding in data:
    print(embedding)

A saída de streaming foi truncada nas últimas 5000 linhas.

        ...,

        [[0.3173287 ]],

        [[0.51532674]],

        [[0.3130854 ]]]], dtype=float32), 'filename': 'XC195760.png', 'latitude': 0.41290341674329145, 'longitude': 0.015001770463884472, 'scientific_name': 'Prinia socialis', 'common_name': 'Ashy Prinia'}
{'path': 'ashpri1/XC197604.png', 'class': 'ashpri1', 'resnet_features': array([[[[0.32327673]],

        [[0.51615936]],

        [[0.45737064]],

        ...,

        [[0.3231569 ]],

        [[0.4404653 ]],

        [[0.32931465]]]], dtype=float32), 'filename': 'XC197604.png', 'latitude': 0.4124438013499508, 'longitude': 0.015031699449911545, 'scientific_name': 'Prinia socialis', 'common_name': 'Ashy Prinia'}
{'path': 'ashpri1/XC551770.png', 'class': 'ashpri1', 'resnet_features': array([[[[0.3680427 ]],

        [[0.5463516 ]],

        [[0.46263507]],

        ...,

        [[0.3311883 ]],

        [[0.50549906]],

        [[0.33130783]]]], dtype=float32), '

In [ ]:
data[0]

{'path': 'ashdro1/XC114598.png',
 'class': 'ashdro1',
 'resnet_features': array([[[[0.3838624 ]],
 
         [[0.51760083]],
 
         [[0.47127506]],
 
         ...,
 
         [[0.34451303]],
 
         [[0.4395803 ]],
 
         [[0.31603485]]]], dtype=float32),
 'filename': 'XC114598.png',
 'latitude': 0.4105609766158399,
 'longitude': 0.016649517860948995,
 'scientific_name': 'Dicrurus leucophaeus',
 'common_name': 'Ashy Drongo'}

Preparando os dados para envio

In [ ]:
print(type(data))  # Deve ser <class 'list'>
print(type(data[0]))  # Deve ser <class 'dict'>
print(data[0].keys())  # Deve mostrar as chaves disponíveis

<class 'list'>
<class 'dict'>
dict_keys(['path', 'class', 'resnet_features', 'filename', 'latitude', 'longitude', 'scientific_name', 'common_name'])


In [ ]:
# Preparar dados para a LSTM
inputs = []
labels = []

for image in data:
    sequence_tensor = torch.tensor(image['resnet_features'].flatten(), dtype=torch.float32)
    features = torch.cat((sequence_tensor, torch.tensor([image['latitude'], image['longitude']], dtype=torch.float32)))
    inputs.append(features)
    labels.append(image['class'])

inputs = torch.stack(inputs)
label_encoder = LabelEncoder()
labels = torch.tensor(label_encoder.fit_transform(labels), dtype=torch.long)

In [ ]:
len(inputs[0])

2050

In [ ]:
inputs[0]

tensor([0.3839, 0.5176, 0.4713,  ..., 0.3160, 0.4106, 0.0166])

Tratando dados nulos com KNN para uma média global e geral

In [ ]:
from sklearn.impute import KNNImputer
features_np = inputs.numpy()
imputer = KNNImputer(n_neighbors=5)
features_imputed = imputer.fit_transform(features_np)
# Converter de volta para tensor PyTorch
inputs = torch.tensor(features_imputed, dtype=torch.float32)

In [ ]:
inputs = inputs.unsqueeze(1)  # Adiciona uma dimensão de sequência de tamanho 1
print(inputs.shape)  # Deve mostrar torch.Size([1048, 1, 2050])

torch.Size([1048, 1, 2050])


In [ ]:
print(f"{type(labels)} e {len(labels)} com {labels.unique()}")

<class 'torch.Tensor'> e 1048 com tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


Separando os dados

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(inputs, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
print(f'X_train: {len(X_train)}, {X_train.shape} e y_train {len(y_train)}, {y_train.shape}')

X_train: 733, torch.Size([733, 1, 2050]) e y_train 733, torch.Size([733])


In [ ]:
print(f'X_val: {len(X_val)}, {X_val.shape} e y_val: {len(y_val)}, {y_val.shape}')

X_val: 157, torch.Size([157, 1, 2050]) e y_val: 157, torch.Size([157])


In [ ]:
print(f'X_test: {len(X_test)}, {X_test.shape} e y_test: {len(y_test)}, {y_test.shape}')

X_test: 158, torch.Size([158, 1, 2050]) e y_test: 158, torch.Size([158])


In [ ]:
if torch.isnan(X_test).any():
    print("Existem NaNs em X_train.")
else:
    print("Todos os NaNs foram tratados.")

Todos os NaNs foram tratados.


###Classificador LSTM

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # Inicializar estados ocultos e de célula
        # Garanta que os estados estão sendo criados com as dimensões corretas
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Passar os dados pelo LSTM
        out, _ = self.lstm(x, (h0, c0))
        # Pegar a saída da última sequência de tempo
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout_prob=0):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_dim, num_classes)
    """
    def forward(self, x):
        # Inicializar estados ocultos e de célula
        # Garanta que os estados estão sendo criados com as dimensões corretas
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Passar os dados pelo LSTM
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        # Pegar a saída da última sequência de tempo
        out = self.fc(out[:, -1, :])
        return out
    """
    def forward(self, x):
        # Inicializar estados ocultos e de célula
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Passar os dados pelo LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Pegar a saída da última sequência de tempo
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out

# Configurações do Modelo
input_dim = 2050  # Número de features
hidden_dim = 128  # Dimensão oculta
num_layers = 2  # Número de camadas LSTM
num_classes = len(torch.unique(labels))  # Suposição do número de classes

In [ ]:
model = LSTMClassifier(input_dim, hidden_dim, num_layers, num_classes)

In [ ]:
# Critério e Otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import copy

# Configuração de Early Stopping
best_val_loss = np.inf
patience = 40
patience_counter = 0
best_model = None

num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    if torch.isnan(loss):
        print("NaN detected in loss during training")
        break

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model = copy.deepcopy(model.state_dict())  # Salva o melhor modelo
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


Epoch [5/1000], Train Loss: 1.6317, Val Loss: 2.0772
Epoch [10/1000], Train Loss: 1.6224, Val Loss: 1.9620
Epoch [15/1000], Train Loss: 1.8218, Val Loss: 2.0358
Epoch [20/1000], Train Loss: 1.7947, Val Loss: 2.0015
Epoch [25/1000], Train Loss: 1.5662, Val Loss: 2.2269
Epoch [30/1000], Train Loss: 1.5876, Val Loss: 2.0204
Epoch [35/1000], Train Loss: 1.6013, Val Loss: 2.0092
Epoch [40/1000], Train Loss: 1.6948, Val Loss: 2.0803
Epoch [45/1000], Train Loss: 1.7153, Val Loss: 2.2206
Epoch [50/1000], Train Loss: 1.4764, Val Loss: 2.4707
Epoch [55/1000], Train Loss: 1.7844, Val Loss: 2.0425
Epoch [60/1000], Train Loss: 1.5791, Val Loss: 2.1548
Epoch [65/1000], Train Loss: 1.6690, Val Loss: 2.2012
Epoch [70/1000], Train Loss: 1.5106, Val Loss: 1.9926
Epoch [75/1000], Train Loss: 1.5568, Val Loss: 1.9328
Early stopping triggered


Avaliação nos dados de teste

In [ ]:
# Avaliação no conjunto de teste
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 2.2800


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs.data, 1)

# Calcular as métricas
accuracy = accuracy_score(y_test.numpy(), predicted.numpy())
precision = precision_score(y_test.numpy(), predicted.numpy(), average='weighted')
recall = recall_score(y_test.numpy(), predicted.numpy(), average='weighted')
f1 = f1_score(y_test.numpy(), predicted.numpy(), average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Reporte detalhado com precision, recall, f1-score por classe
print("\nClassification Report:\n", classification_report(y_test.numpy(), predicted.numpy()))

Accuracy: 0.1835
Precision: 0.3526
Recall: 0.1835
F1 Score: 0.1499

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        18
           2       0.11      0.31      0.17        13
           3       0.37      0.33      0.35        21
           4       1.00      0.04      0.08        23
           5       1.00      0.17      0.29        18
           6       0.08      0.86      0.14         7
           7       0.38      0.62      0.47        13
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00        11

    accuracy                           0.18       158
   macro avg       0.29      0.23      0.15       158
weighted avg       0.35      0.18      0.15       158



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13